In [1]:
import numpy as np
import numpy.linalg as npl
import numpy.fft as fft
import scipy as sp
import scipy.linalg as spl
import scipy.sparse as sps
import matplotlib.pyplot as plt
#%matplotlib inline
%matplotlib widget

pi = np.pi

# Exercice 1 : Equation de transport

$$u^{n+1}_{j} = u^{n-1}_j + c \frac{\partial t}{\partial x} (u^n_{j+1} - u^n_{j-1})$$

In [2]:
def u0(x, mu=0, sigma2=1):
    exp = np.exp(-0.5 * ((x-mu)**2)/sigma2)
    return exp/np.sqrt(2*pi*sigma2)

In [3]:
def v0(x, k=0.5, mu=0, sigma2=1):
    return u0(x, mu, sigma2) * np.sin(2*k*pi*x)

In [4]:
def w0(x, a=-2.5, b=2.5):
    return np.where((x>=a) & (x<=b), 1, 0) 

In [5]:
def sch_Saute_Mouton(U0, U1, c, dt, dx, T):
    U = [U0, U1]
    N = U0.size
    A = c*dt/dx * sps.diags([1, -1, 1, -1], [-N, -1, 1, N], shape=[N, N]).toarray() 
    
    n = 2
    
    while n*dt <= T:
        U2 = U[-2] - A.dot(U[-1])
        U.append(U2)
        n += 1
    
    return U

In [6]:
def sch_décentré_amont(U0, c, dt, dx, T):
    U = [U0]
    N = U0.size
    A = np.eye(N) - c*dt/(dx) * sps.diags([-1, 1, -1], [-1, 0, N-1], shape=[N, N]).toarray()
    
    n = 1
    
    while n*dt <= T:
        U1 = A.dot(U[-1])
        U.append(U1)
        n += 1
        
    return U

In [7]:
def test_sch_décentré_amont(min, max, c, dt=0.005, dx=0.01, T=0.5):
    x = np.linspace(min, max, int((max-min)/dx))
    for func in [u0, v0, w0]:
        U = sch_décentré_amont(func(x), c, dt, dx, T)
        plt.figure(clear=True)
        plt.plot(x ,U[0])
        plt.plot(x, U[-1])
        plt.show()

In [14]:
test_sch_décentré_amont(-5, 5, c=0.5, dt=0.01, dx=0.01, T=60)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/pjbenard/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
def sch_décentré(U0, c, dt, dx, T):
    U = [U0]
    N = U0.size
    A = np.eye(N) + c*dt/dx * sps.diags([-0.5, 1.5, -1, -0.5, 1.5], [-N+1, -1, 0, 1, N-1], shape=[N, N]).toarray()
    
    n = 1
    
    while n*dt <= T:
        U1 = A.dot(U[-1])
        U.append(U1)
        n += 1
        
    return U

In [16]:
def test_sch_décentré(min, max, c, dt=0.005, dx=0.01, T=0.5):
    x = np.linspace(min, max, int((max-min)/dx))
    for func in [u0, v0, w0]:
        U = sch_décentré(func(x), c, dt, dx, T)
        plt.figure(clear=True)
        plt.plot(x ,U[0])
        plt.plot(x, U[-1])
        plt.show()

In [23]:
test_sch_décentré(-5, 5, c=0.5, dt=0.005, dx=0.01, T=40)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plt.close('all')

## Exercice 2 : Visualiser les fonctions d'un espace de Sobolev

On travaille avec les espaces suivants :

$$L^2\left(\left[0, 2\pi\right]^d\right) = \left\{ f, \int_{0}^{2\pi} \left|f(x)\right|^2 dx < +\infty \right\}$$ 

$$H^{m}_{per}\left(\left[0, 2\pi\right]\right) = 
\left\{ 
f \in L^2\left(\left[0, 2\pi\right]\right), \sum_{k \in \mathbb{z}} \left(1 + k^{2}\right)^m \left|c_{k}(f)\right|^2 < +\infty 
\right\}$$

$$H^{m}_{per}\left(\left[0, 2\pi\right]^2\right) = 
\left\{ 
f \in L^2\left(\left[0, 2\pi\right]^2\right), \sum_{k, l \in \mathbb{z}} \left(1 + k^{2} + l^{2}\right)^m \left|c_{k,l}(f)\right|^2 < +\infty 
\right\}$$

On veut constuire des fonctions $f_{i}$ dont les coefficients de Fourier vérifient :
$$\left|c_{k}\left(f_{i}\right)\right| = \left(1 + k^{2}\right)^{-\alpha/2} \text{ en dimension 1}$$
$$\left|c_{k, l}\left(f_{i}\right)\right| = \left(1 + k^{2} + l^{2}\right)^{-\alpha/2} \text{ en dimension 2}$$

En cherchant $f_{2}$ telle que $f_{2} \in H^{1}([0, 2\pi]^{d}) \text{ et} \notin H^{2}([0, 2\pi]^{d})$, on obtient :

$$
\sum_{k \in \mathbb{z}} \left(1 + k^{2}\right)^m \left|c_{k}(f)\right|^2 < +\infty 

\sum_{k, l \in \mathbb{z}} \left(1 + k^{2} + l^{2}\right)^m \left|c_{k,l}(f)\right|^2 < +\infty 

$$

In [24]:
def sobolev(alpha=1, N=2**9, pha=None):
    K, L = np.meshgrid(np.arange(N), np.arange(N))
    module = (1 + K**2 + L**2)**(-alpha/2)
    
    if pha is None:
        phase = np.random.random((N, N))*2*pi
        coeffs = module * np.exp(1j * phase)
    
    coeffs = module * np.exp(1j * pha)
    f = fft.ifft2(coeffs)
    
    return np.real(f)

In [25]:
fig = plt.figure(figsize=(12, 12))
N = 2**9
seed = np.random.randint(10000)
np.random.seed()
phase = np.random.random((N, N))*2*pi
for i in range(1, 5):
    
    ax = fig.add_subplot(220+i)
    image = sobolev(i, N, phase)
    ax.imshow(image)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

phase = np.random.random((N, N))*2*pi
X, Y = np.meshgrid(np.arange(N), np.arange(N))

image = sobolev(10, N, phase)
    
ax.plot_surface(X, Y, image, cmap="Purples")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
images = []

seed = np.random.randint(10000)
np.random.seed()
phase = np.random.random((N, N))*2*pi
for i in np.linspace(1, 5, num=50):
    image = sobolev(i, N, phase)
    images.append(image)

In [30]:
len(images)
plt.figure()
plt.imshow(images[0])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
def plot_sobolev(alpha=1, seed=100):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    N = 2**7
    np.random.seed(seed)
    phase = np.random.random((N, N))*2*pi

    image = sobolev(alpha, N, phase)
    
    ax.imshow(image)
    plt.show()

In [31]:
plt.close('all')

## Exercice 3